# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [29]:
# initial imports
import os
import pandas as pd
import panel as pn
import plotly.express as px
import plotly.graph_objects as go
import holoviews as hv
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

In [30]:
pn.extension("plotly")

In [31]:
import hvplot.pandas
import matplotlib.pyplot as plt

In [32]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [33]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path2 = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path2)

#Second dataframe, without year as index.
sfo_data_new = pd.read_csv(file_path)

- - -

In [34]:
# Define Panel Visualization Functions

#####################################################

# The pandas/matplotlib plots were commented out,
# Because they didn't load into the panels as planned.
# Plotly Express was used instead.

#####################################################

def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_housing_units = sfo_data.groupby('year')['housing_units'].mean()
    #sfo_housing_plot = sfo_housing_units.plot(kind='bar',ylim=(369000,387750),
                                          #figsize=(8,5),
                                          #title='Housing Units in San Francisco from 2010 to 2016',)
    #sfo_housing_plot.set_ylabel('Housing Units')
    #sfo_housing_plot.set_xlabel('Year')
    
    return px.bar(sfo_housing_units, range_y=('360000','390000'),
                  labels={'year':'Year','value':'Housing Units'})

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_avg_gross_rent = sfo_data.groupby('year')['sale_price_sqr_foot','gross_rent'].mean().drop(columns='sale_price_sqr_foot')
    #sfo_gross_plot = sfo_avg_gross_rent['gross_rent'].plot(title='Average Gross Rent in San Francisco')
    #sfo_gross_plot.set_ylabel('Gross Rent')
    #sfo_gross_plot.set_xlabel('Year')
    
    return px.line(sfo_avg_gross_rent,labels={'year':'Year','value':'Gross Rent'})

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_sale_sqr_foot = sfo_data.groupby('year')['sale_price_sqr_foot'].mean()
    #sfo_sale_sqr_foot_plot = sfo_sale_sqr_foot.plot(title='Average Sale Price per Square Foot in San Francisco')
    #sfo_sale_sqr_foot_plot.set_xlabel('Year')
    #sfo_sale_sqr_foot_plot.set_ylabel('Avg. Sale Price')
    
    return px.line(sfo_sale_sqr_foot, labels={'year':'Year','value':'Sale Price per Square Foot'})

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sfo_hood_avg = sfo_data.groupby(['year','neighborhood']).mean().reset_index()
    sfo_hood_avg_plot = sfo_hood_avg.hvplot.line(x='year',y='sale_price_sqr_foot',
                                                 groupby='neighborhood',
                                                 xlabel='Year',
                                                 ylabel='Avg. Sale Price per Square Foot')
    
    return sfo_hood_avg_plot

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    sfo_top_10_exp = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False).reset_index().head(10)
    sfo_top_10_plot = sfo_top_10_exp.hvplot.bar(x='neighborhood',y='sale_price_sqr_foot',rot=90,
                                                title='Top 10 Expensive Neighborhoods in SFO',
                                                frame_height=200,frame_width=500,xlabel='Neighborhood',
                                                ylabel='Avg. Sale Price per Square Foot')
    
    return sfo_top_10_plot

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    sfo_top_10_exp = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False).reset_index().head(10)    
    sfo_top10_coord_plot = px.parallel_coordinates(sfo_top_10_exp, color='sale_price_sqr_foot')
    
    return sfo_top10_coord_plot


def parallel_categories():
    """Parallel Categories Plot."""
    
    sfo_top_10_exp = sfo_data.groupby(['neighborhood']).mean().sort_values(by='sale_price_sqr_foot', ascending=False).reset_index().head(10)
    sfo_top10_cat_plot = px.parallel_categories(sfo_top_10_exp, color='sale_price_sqr_foot')
    
    return sfo_top10_cat_plot


def neighborhood_map():
    """Neighborhood Map"""
    
    sfo_loc = df_neighborhood_locations
    sfo_loc.head()
    
    sfo_mean = sfo_data.groupby(['neighborhood']).mean().reset_index()
    sfo_mean.head()
    
    sfo_comb = pd.concat([sfo_loc,sfo_mean], axis=1)
    sfo_comb.drop(columns=['neighborhood'], inplace=True)
    sfo_comb.head()
    
    sfo_hood_map = px.scatter_mapbox(sfo_comb, lat='Lat',lon='Lon',color="gross_rent",
                                     size="sale_price_sqr_foot",
                                     title='Average Sale Price Per Square Foot and Gross Rent in San Francisco',
                                     color_continuous_scale=px.colors.cyclical.IceFire)
    
    return sfo_hood_map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [35]:
# Creating Columns/Rows
sfo_column_one = pn.Column("##### This dashboard presents a visual analysis of historical prices of house units, sale price per square foot and gross rent in San Francisco, California from 2010 to 2016. You can navigate through the tabs above to explore more details about the evolution of the real estate market on The Golden City across these years.",
                           neighborhood_map()
                          )
sfo_row_two = pn.Column("##### Yearly Market Analysis",
                     housing_units_per_year(),
                     average_gross_rent(),
                     average_sales_price()
                    )
sfo_column_three = pn.Column("##### Interactive Neighborhood Pane\nTop 10 Most Expensive Neighborhoods",
                             average_price_by_neighborhood(),
                             top_most_expensive_neighborhoods()
                            )
sfo_column_four = pn.Column("##### Parallel Plots",
                            parallel_coordinates(),
                            parallel_categories()
                           )

# Creating tabs
sfo_dashboard = pn.Tabs(
    ("Welcome", sfo_column_one),
    ("Yearly Market Analysis", sfo_row_two),
    ("Neighborhood Analysis", sfo_column_three),
    ("Parallel Plots Analysis", sfo_column_four)
)

sfo_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
        [3] Plotly(Figure)
    [2] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [2] HoloViews(Bars)
    [3] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)

## Serve the Panel Dashboard

In [37]:
sfo_dashboard.servable()